# Borrower Reliability Study

The customer is the credit department of the bank. It is necessary to find out whether the marital status and the number of children of the client affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the study will be taken into account when building a **credit scoring** model - a special system that assesses the ability of a potential borrower to repay a loan to a bank.

According to the data documentation:

* `children` - the number of children in the family;
* `days_employed` - total work experience in days;
* `dob_years` — client's age in years;
* `education` - the level of education of the client;
* `education_id` — education level identifier;
* `family_status` - marital status;
* `family_status_id` - marital status identifier;
* `gender` — gender of the client;
* `income_type` - employment type;
* `debt` - whether he had a debt to repay loans;
* `total_income` - monthly income;
* `purpose` - the purpose of the loan.

## Step 1. Explore the data

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Conclusion**

The table has 12 columns. Column data types: `int64`, `float64` and `object`.

There are also incorrect values in the data:
* The number of values in the columns varies, so there are missing values
* There are negative and too large values in the `children` and `days_employed` columns
* Incorrect case of values in the `education` column
* Too many similar purposes for obtaining a loan

Each row of the table contains the data of the bank's clients. Some of the columns describe the client and his family: number of children, length of service, age, level of education, marital status, gender, type of employment and monthly income. The other part is related to the client's credit history: whether he had a debt on a loan and the purpose of obtaining a loan.
Preliminarily, it can be said that there is enough data to test hypotheses. But there are gaps in the data.

To move forward, problems in data have to be fixed. 

## Step 2. Data preprocessing

First, let's calculate how many missing values are in the table.

In [4]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Conclusion**

In the data, we omit the values of the total work experience in days and monthly income. Obviously, the monthly income directly depends on the length of service, so the data in the columns is not displayed accordingly.

Let us replace the monthly income data with the median value for each type of employment.

In [5]:
employers_types = ['сотрудник', 'пенсионер', 'компаньон', 'госслужащий']
for empoloyer_type in employers_types:   
    median_income = df.loc[df['income_type'] == empoloyer_type]['total_income'].median()
    print(median_income)
    df.loc[(df['total_income'].isna()) & (df['income_type'] == empoloyer_type), 'total_income'] = median_income

142594.39684740017
118514.48641164352
172357.95096577113
150447.9352830068


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


There is one missing value in the `total_income` column, let's find it.

In [8]:
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости


In [9]:
# Find all entrepreneurs in the table by the `income_type` column
df[df['income_type'] == 'предприниматель']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5936,0,NaN,58,высшее,0,женат / замужем,0,M,предприниматель,0,NaN,покупка жилой недвижимости
18697,0,-520.848083,27,высшее,0,гражданский брак,1,F,предприниматель,0,499163.144947,на проведение свадьбы


There are only 2 of them, so the median among entrepreneurs cannot be calculated.

A client with missing data has no children, as well as another entrepreneur, so this person will not be an indicator for us. From this follows the decision to remove the row from the entire table in order to correctly analyze the data.

In [10]:
df = df.dropna(subset=['total_income'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  float64
 11  purpose           21524 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [11]:
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118514.486412,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514.486412,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,172357.950966,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,142594.396847,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,172357.950966,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,строительство жилой недвижимости


**Conclusion**

During the skip processing step, we found gaps in the `total_income` and `days_employed` columns. We filled in the data in the `total_income` column with median data, while the data in `days_employed` could not be filled in.

In [11]:
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21524 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21524 non-null  int64  
 3   education         21524 non-null  object 
 4   education_id      21524 non-null  int64  
 5   family_status     21524 non-null  object 
 6   family_status_id  21524 non-null  int64  
 7   gender            21524 non-null  object 
 8   income_type       21524 non-null  object 
 9   debt              21524 non-null  int64  
 10  total_income      21524 non-null  int64  
 11  purpose           21524 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB


### Duplicate Handling

In [12]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [13]:
df['education'] = df['education'].str.lower()
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [27]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

The `purpose` column contains a large number of similar values, for this we need to bring all the values to similar ones, for this we will write a function in which we lemmatize the data.

Let's create a new column called `purpose_general`, which will store 4 unique values: `real estate`, `education`, `wedding`, `wedding`.


In [14]:
from pymystem3 import Mystem
m = Mystem()
def unique_purpose(purpose):
    lemmas = ' '.join(m.lemmatize(purpose))
    for lemma in lemmas:
        if 'жилье' in lemmas or 'недвижимость' in lemmas:
            return 'недвижимость'
        elif 'образование' in lemmas:
            return 'образование'
        elif 'свадьба' in lemmas:
            return 'свадьба'
        elif 'автомобиль' in lemmas:
            return 'автомобиль'
df['purpose_general'] = df['purpose'].apply(unique_purpose)
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_general
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [15]:
df.duplicated().sum()

71

At this stage, we have 71 duplicates, we will remove them by resetting the index

In [16]:
df = df.drop_duplicates().reset_index(drop=True)

In [17]:
df.duplicated().sum()

0

In [18]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

There are negative and oversized values (-1 and 20) in the values. Most likely there was an error during data entry and the person meant 1 and 2, so we will replace these values accordingly

In [19]:
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [20]:
df['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [21]:
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

The values in this column are very chaotic. There are a variety of negative values, as well as non-integer ones. In this case, it is more convenient to count the length of service in years.

Let's remove negative values first.

In [22]:
df['days_employed'] = df['days_employed'].abs()

In [23]:
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Next, let's try to create a new column with years of service, take 365 days for a year and round the numbers.

In [24]:
df['years_employed'] = df['days_employed'] / 365

In [25]:
df['years_employed'].describe()

count    19351.000000
mean       183.328024
std        380.906522
min          0.066141
25%          2.539751
50%          6.011563
75%         15.172281
max       1100.699727
Name: years_employed, dtype: float64

Very strange values. Lots of people with over 100 years of experience. The maximum number reaches 1100 years.

In [26]:
df['total_income'].describe()

count    2.145300e+04
mean     1.653040e+05
std      9.816313e+04
min      2.066700e+04
25%      1.076200e+05
50%      1.425940e+05
75%      1.958180e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [27]:
df['total_income'] = df['total_income'].astype(int)
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_general,years_employed
12368,2,992.122926,31,неоконченное высшее,2,женат / замужем,0,F,компаньон,0,141671,приобретение автомобиля,автомобиль,2.718145
477,2,2837.717108,41,высшее,0,женат / замужем,0,F,госслужащий,0,99015,жилье,недвижимость,7.774567
20577,2,1223.172066,32,среднее,1,гражданский брак,1,M,компаньон,0,356590,на проведение свадьбы,свадьба,3.351156
7696,0,369845.413661,51,среднее,1,гражданский брак,1,F,пенсионер,0,390646,на проведение свадьбы,свадьба,1013.275106
17277,0,395933.742936,57,среднее,1,женат / замужем,0,F,пенсионер,0,176668,операции с жильем,недвижимость,1084.749981


### Data categorization

In [28]:
df[['education_id', 'education']].drop_duplicates()

,education_id,education
0,0,высшее
1,1,среднее
13,2,неоконченное высшее
31,3,начальное
2962,4,ученая степень


In [29]:
df[['family_status_id', 'family_status']].drop_duplicates()

,family_status_id,family_status
0,0,женат / замужем
4,1,гражданский брак
18,2,вдовец / вдова
19,3,в разводе
24,4,Не женат / не замужем


## Step 3: Answer the questions

- Is there a dependence between having children and repaying a loan on time?

In [30]:
children_reliance = df.groupby('children')['debt'].sum() / df.groupby('children')['debt'].count() * 100
children_reliance.sort_values()

children
5    0.000000
0    7.544358
3    8.181818
1    9.165808
2    9.492481
4    9.756098
Name: debt, dtype: float64

In [31]:
children_table = df.pivot_table(index = ['children'],
                                columns = ['debt'],
                                values = 'purpose', aggfunc='count')

children_table['average'] = children_table[1] / children_table[0] * 100
children_table

debt,0,1,average
children,,,
0,13027.0,1063.0,8.159975
1,4410.0,445.0,10.090703
2,1926.0,202.0,10.488058
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,NaN,NaN


**Conclusion**

Based on the data, it can be concluded that children do not particularly affect the repayment of the loan. Childless people, as well as people with 3 children, repay the loan the best, we have almost no data for people with 5 children and the indicator is at zero. And the worst rate of return of loans in people is people who have 4 and 2 children.

-Is there a dependence between marital status and loan repayment on time?

In [32]:
family_reliance = df.groupby('family_status')['debt'].sum() / df.groupby('family_status')['debt'].count() * 100
family_reliance.sort_values()

family_status
вдовец / вдова           6.569343
в разводе                7.112971
женат / замужем          7.545793
гражданский брак         9.347145
Не женат / не замужем    9.750890
Name: debt, dtype: float64

In [33]:
family_table = df.pivot_table(index=['family_status'], columns=['debt'], values='total_income', aggfunc='count')
family_table['average'] = family_table[1] / family_table[0] * 100
family_table

debt,0,1,average
family_status,,,
Не женат / не замужем,2536,274,10.804416
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3763,388,10.310922
женат / замужем,11407,931,8.161655


**Conclusion**

Based on these data, we can conclude that people who are divorced and widowed are the best at repaying loans, while the rates of single people and people in a civil marriage are low.

- Is there a correlation between income level and loan repayment on time?

It is not entirely clear what limits to give to income levels, so I will guess and draw the limits myself. We will divide income levels into 4 categories: `low`, `medium`, `high` and `very high`.

In [34]:
df['total_income'].quantile([0.25,0.5,0.75])

0.25    107620.0
0.50    142594.0
0.75    195818.0
Name: total_income, dtype: float64

In [35]:
def how_rich(total_income):
    if total_income <= 107620.0:
            return 'low'
    if total_income <= 142594.0:
            return 'medium'
    if total_income < 195818.0:
            return 'high'
    return 'very high'

In [36]:
df['how_rich'] = df['total_income'].apply(how_rich)
df.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_general,years_employed,how_rich
10318,0,6926.159986,43,среднее,1,Не женат / не замужем,4,F,сотрудник,0,69850,операции с недвижимостью,недвижимость,18.975781,low
13668,0,2202.767385,29,высшее,0,Не женат / не замужем,4,F,компаньон,0,206928,покупка своего жилья,недвижимость,6.034979,very high
1234,0,1774.256890,41,среднее,1,гражданский брак,1,F,сотрудник,0,267235,сыграть свадьбу,свадьба,4.860978,very high
7985,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба,свадьба,NaN,medium
12954,1,5034.339776,43,среднее,1,женат / замужем,0,M,сотрудник,0,144584,заняться высшим образованием,образование,13.792712,high


In [37]:
income_reliance = df.groupby('how_rich')['debt'].sum() / df.groupby('how_rich')['debt'].count() * 100
income_reliance.sort_values()

how_rich
very high    7.140194
low          7.960477
high         8.539840
medium       8.815477
Name: debt, dtype: float64

In [38]:
income_table = df.pivot_table(index=['how_rich'], columns=['debt'], values='education_id', aggfunc='count')
income_table['average'] = income_table[1] / income_table[0] * 100
income_table

debt,0,1,average
how_rich,,,
high,4798,448,9.337224
low,4937,427,8.648977
medium,4996,483,9.667734
very high,4981,383,7.689219


**Conclusion**

Based on the results obtained, it can be concluded that people with medium and high income levels are the longest to repay the loan on time. Although the difference is small, it is still there. The highest return rates are associated with people with very high income levels.

- How do different purposes of a loan affect its repayment on time?

In [39]:
purpose_reliance = df.groupby('purpose_general')['debt'].sum() / df.groupby('purpose_general')['debt'].count() * 100
purpose_reliance.sort_values()

purpose_general
недвижимость    7.234043
свадьба         8.003442
образование     9.220035
автомобиль      9.359034
Name: debt, dtype: float64

In [40]:
purpose_table = df.pivot_table(index=['purpose_general'], columns=['debt'], values='education_id', aggfunc='count')
purpose_table['average'] = purpose_table[1] / purpose_table[0] * 100
purpose_table

debt,0,1,average
purpose_general,,,
автомобиль,3903,403,10.325391
недвижимость,10028,782,7.798165
образование,3643,370,10.156464
свадьба,2138,186,8.699719


**Conlusion**

This finding suggests that people whose loans are related to real estate or marriage pay off loans much faster than those who took out a loan for education or a car.

## Step 4. General conclusion

We did a lot of analysis based on data from the bank. The main goal was to understand how different factors influence loan repayment on time. The main question was: "Does the marital status and the number of children of the client affect the fact of repaying the loan on time?"

Having dealt with the errors in the data and after analyzing them, I can give a final answer to the question posed.

The difference in indicators of marital status is quite obvious. Attention is drawn to single people and people in a civil marriage, whose indicators are the lowest.

Data related to the number of children is not so obvious. We cannot draw explicit statistics in them. Based on the relationships, it can be seen that people who have 1, 2 or 4 children repay the loan the worst. This is most likely due to education loans and loans for large cars for large families. Unfortunately, we do not have enough data on people with 5 children, so we were not able to draw statistics on them.